<a href="https://colab.research.google.com/github/kirath2205/Final-Year-Project/blob/main/defensive_distillation_vgg16_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install numba 

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [16]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D , UpSampling3D , Lambda , Conv2D ,MaxPooling2D
from tensorflow.keras import optimizers
from keras.datasets import cifar100,cifar10,fashion_mnist
import tensorflow as tf
from keras.utils import np_utils
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input , decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
tf.keras.backend.clear_session()

AttributeError: ignored

In [19]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.2,
        temperature=20,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

    def call(self, inputs, *args, **kwargs):
       return self.student(inputs)

In [20]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [21]:
def select_dataset(index=1): #1 for cifar10 , 2 for cifar100 , 3 for fashion mnist
  if(index == 1):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    channel = 3
    num_classes = 10
  if(index == 2):
    (x_train, y_train), (x_test, y_test) = cifar100.load_data()
    num_classes = 100
    channel = 3
  if(index == 3):
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    x_train = x_train.reshape((60000, 28, 28, 1))
    x_test =  x_test.reshape((10000, 28, 28, 1))
    num_classes = 10
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)
    channel = 1
    return (x_train , y_train , x_test , y_test , num_classes , channel)

  #Pre-process the data
  x_train = preprocess_input(x_train)
  x_test = preprocess_input(x_test)
  datagen = ImageDataGenerator(preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=True))
  datagen.fit(x_train)
  y_train = np_utils.to_categorical(y_train, num_classes)
  y_test = np_utils.to_categorical(y_test, num_classes)

  return (x_train , y_train , x_test , y_test , num_classes , channel , datagen)

In [22]:
index = 1
x_train , y_train , x_test , y_test , num_classes ,channel , datagen = select_dataset(index)
model_name = 'desktop/Trained_models/defensive_distillation_keras_resnet50_cifar10'
model_path = 'desktop/Trained_models/defensive_distillation_keras_resnet50_cifar10.h5'

In [18]:
index = 2
x_train , y_train , x_test , y_test , num_classes ,channel , datagen = select_dataset(index)
model_name = 'desktop/Trained_models/defensive_distillation_keras_resnet50_cifar100'
model_path = 'desktop/Trained_models/defensive_distillation_keras_resnet50_cifar100.h5'

In [23]:
temp = 200
weight_decay = 0.0005
if(index==1 or index==2):
  teacher = keras.Sequential(
      [
        layers.UpSampling2D((5,5)),
        layers.Conv2D(64,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay),input_shape = (160,160,3)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(64,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),
        layers.Conv2D(128,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(128,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),
        layers.Conv2D(256,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(256,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(256,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(512,kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Dropout(0.25),
        layers.Dense(num_classes),
        layers.BatchNormalization(),
        layers.Lambda((lambda x: x / temp)),
      ],
      name="teacher",
  )

# Create the student
  student = keras.Sequential(
      [
        layers.UpSampling2D((5,5)),
        layers.Conv2D(64,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay),input_shape = (160,160,3)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(64,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),
        layers.Conv2D(128,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(128,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),
        layers.Conv2D(256,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(256,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(256,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(512,3,padding="same",kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),
        layers.Flatten(),
        layers.Dense(512,kernel_regularizer=regularizers.l2(weight_decay)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Dropout(0.25),
        layers.Dense(num_classes),
        layers.BatchNormalization(),
        layers.Lambda((lambda x: x / temp)),
      ],
      name="student",
  )


In [ ]:
epochs = 140
learning_rate = 0.1
lr_decay = 1e-6
lr_drop = 20
callbacks = [ 
    tf.keras.callbacks.ReduceLROnPlateau( factor = 0.7, patience = 5, min_lr = 0.000001, verbose = 1,monitor='val_loss' )
  ]
batch_size = 128
teacher.compile(
    optimizer=keras.optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

# Train and evaluate teacher on data.
history_teacher = teacher.fit_generator(datagen.flow(x_train, y_train,
                                  batch_size=batch_size),
                                  steps_per_epoch=x_train.shape[0] // batch_size,
                                  epochs=epochs,
                                  validation_data=(x_test, y_test),callbacks=callbacks)
teacher.evaluate(x_test, y_test)

In [ ]:
tf.keras.backend.clear_session()
import matplotlib.pyplot as plt

plt.figure(figsize = (9,6))
plt.plot(history_teacher.history['val_loss'],label='Test loss')
plt.plot(history_teacher.history['loss'],label='Train loss')
plt.title('Loss curve for resnet model')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

# Plot history: Accuracy
plt.figure(figsize = (9,6))
plt.plot(history_teacher.history['val_categorical_accuracy'],label = 'Test accuracy')
plt.plot(history_teacher.history['categorical_accuracy'],label = 'Train accuracy')
plt.title('Accuracy curve for resnet model')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()


In [ ]:

distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.2,
    temperature=200,
)
callbacks = [ 
    #tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1, save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau( factor = 0.7, patience = 5, min_lr = 0.000001, verbose = 1, monitor='val_student_loss'), #patience = 7 and 20 for cifar-100 , patience = 5 and 10 for cifar-10
    #tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy' , patience = 10)
  ]
# Distill teacher to student
history_distiller = distiller.fit_generator(datagen.flow(x_train, y_train,
                                  batch_size=batch_size),
                                  steps_per_epoch=x_train.shape[0] // batch_size,
                                  epochs=100,
                                  validation_data=(x_test, y_test),
                                  callbacks = callbacks)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

In [ ]:

import matplotlib.pyplot as plt

plt.figure(figsize = (9,6))
plt.plot(history_distiller.history['val_student_loss'],label='Test loss')
plt.plot(history_distiller.history['student_loss'],label='Train loss')
plt.title('Loss curve for resnet model')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

# Plot history: Accuracy
plt.figure(figsize = (9,6))
plt.plot(history_distiller.history['val_categorical_accuracy'],label = 'Test accuracy')
plt.plot(history_distiller.history['categorical_accuracy'],label = 'Train accuracy')
plt.title('Accuracy curve for resnet model')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()


In [ ]:


def make_prediction(model , image , true_value):
  true_label_index = -1
  for i in range(len(true_value)):

    if(true_value[i]==1):
      true_label_index = i
      break

  prediction = model.predict(image)[0]
  probability = float('-inf')
  predicted_label_index = -1

  for i in range(len(prediction)):

    if(prediction[i]>probability):
      probability = prediction[i]
      predicted_label_index = i

  if(true_label_index!=predicted_label_index):
    return 1
  return 0



In [ ]:
import keras,os
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D , Dropout
import numpy as np
from tensorflow import keras
from keras.initializers import glorot_uniform
from keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input
from keras import regularizers
from absl import app, flags

# Install bleeding edge version of cleverhans
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method

In [ ]:
from keras.preprocessing.image import img_to_array

epsilon_end = 0.8
#logits_model = tf.keras.Model(model.input,model.layers[-1].output)
total_images = 200

result={}
epsilon = 0.1
while(epsilon <= epsilon_end):

  fgsm_counter = 0
  pgd_counter = 0
  print("Epsilon value - ",epsilon)
  print()
  for image_index in range(total_images):
  
  
    image = x_test[image_index]
    image = img_to_array(image)
    image = image.reshape(1, 32, 32, 3)
    true_value = y_test[image_index]
    original_prediction = make_prediction(distiller,image,true_value)
    fgsm_sample = fast_gradient_method(distiller, image, epsilon, np.inf, targeted=False)
    pgd_sample = projected_gradient_descent(distiller, image, epsilon, 0.01, 40, np.inf)
    print(epsilon,' ',image_index)
    fgsm_prediction = make_prediction(distiller , fgsm_sample , true_value)
    pgd_prediction = make_prediction(distiller , pgd_sample , true_value)
    pgd_counter+=pgd_prediction
    fgsm_counter+=fgsm_prediction
    print(pgd_counter,' ',fgsm_counter)
  result[epsilon] = (fgsm_counter , pgd_counter)
  epsilon+=0.1


In [ ]:
for key in result.keys():
  print(key,' ',result[key])